# Pneumonia



---


> ***Pneumonia*** is a lung infection that causes inflammation and damage to lung tissue.In a healthy person, the alveoli fill with air during breathing; however, in people with pneumonia, the alveoli are already filled with pus and fluid that affects gas exchange.


---





In [ ]:
from IPython.display import Image 
Image(url="https://www.clinicbarcelona.org/uploads/media/default/0001/09/028fd0195b02b208a3118ea9797c3b2e37ed148f.png")

# What is Pneumonia?



---


***Pneumonia*** is an inflammatory condition of the lung affecting primarily the small air sacs known as alveoli.Symptoms typically include some combination of productive or dry cough, chest pain, fever and difficulty breathing. The severity of the condition is variable. ***Pneumonia*** is usually caused by infection with viruses or bacteria and less commonly by other microorganisms, certain medications or conditions such as autoimmune diseases.Risk factors include cystic fibrosis, chronic obstructive pulmonary disease (COPD), asthma, diabetes, heart failure, a history of smoking, a poor ability to cough such as following a stroke and a weak immune system. Diagnosis is often based on symptoms and physical examination.
Chest X-ray, blood tests, and culture of the sputum may help confirm the diagnosis.The disease may be classified by where it was acquired, such as community- or hospital-acquired or healthcare-associated pneumonia.


---




# package

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
import numpy as np # linear algebra
import pandas as pd 
from keras.models import Model

# Description of the Pneumonia Dataset



---


The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal). Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care. For the analysis of chest x-ray images, all chest radiographs were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.


---



In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
#on adapte notre json file de notre kaggle profile
files.upload()

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!ls ~/.kaggle

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#kaggle data
!kaggle datasets download  "paultimothymooney/chest-xray-pneumonia"

In [ ]:
#le fichier est zip , on va unsiper le fichier pour acceder anotre data
!unzip chest-xray-pneumonia.zip

## Loading the Dataset

In [ ]:
train_dir = "/content/chest_xray/chest_xray/train"
test_dir = "/content/chest_xray/chest_xray/test"
val_dir = "/content/chest_xray/chest_xray/val"

print("Train set:\n========================================")
num_pneumonia = len(os.listdir(os.path.join(train_dir, 'PNEUMONIA')))
num_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))
print(f"PNEUMONIA={num_pneumonia}")
print(f"NORMAL={num_normal}")

print("Test set:\n========================================")
print(f"PNEUMONIA={len(os.listdir(os.path.join(test_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(test_dir, 'NORMAL')))}")

print("Validation set:\n========================================")
print(f"PNEUMONIA={len(os.listdir(os.path.join(val_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(val_dir, 'NORMAL')))}")



## Data Visualization & Preprocessing

In [ ]:
pneumonia = os.listdir("/content/chest_xray/train/PNEUMONIA")
pneumonia_dir = "/content/chest_xray/train/PNEUMONIA"

plt.figure(figsize=(20, 10))

for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(pneumonia_dir, pneumonia[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
plt.tight_layout()

***normal cas***

In [ ]:
path = "/content/chest_xray/chest_xray/train/NORMAL"
#listdir =retourne une liste contenant les noms des entrées dans le répertoire donné par chemin
data_exemple = os.listdir(path)
plt.figure(figsize=(15, 10))

for i in range(12):
    plt.subplot(4, 4, i + 1)
    #imread =charge une image à partir du fichier spécifié
    img = plt.imread(os.path.join(path, data_exemple[i]))
    #imshow = affiche l'imagerie 
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
plt.tight_layout()

***cas pneumonia***

In [ ]:
path = "/content/chest_xray/chest_xray/train/PNEUMONIA"
#listdir =retourne une liste contenant les noms des entrées dans le répertoire donné par chemin
data_exemple = os.listdir(path)
plt.figure(figsize=(15, 10))

for i in range(12):
    plt.subplot(4, 4, i + 1)
    #imread =charge une image à partir du fichier spécifié
    img = plt.imread(os.path.join(path, data_exemple[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
plt.tight_layout()

## pixel value distribution

In [ ]:
normal_img = os.listdir("/content/chest_xray/chest_xray/train/PNEUMONIA")[58]
normal_dir = "/content/chest_xray/chest_xray/train/PNEUMONIA"
sample_img = plt.imread(os.path.join(normal_dir, normal_img))
sns.distplot(sample_img.ravel(),
            label=f"Pixel Mean {np.mean(sample_img):.4f} & Standard Deviation {np.std(sample_img):.4f}", kde=False)
plt.legend(loc='upper center')
plt.title('Distribution of Pixel Intensities in the Image')
plt.xlabel('Pixel Intensity')
plt.ylabel('# Pixels in Image')

# ***Deep Learning in Healthcare*** — X-Ray Imaging

## Image Preprocessing

Before training, you'll first modify your images to be better suited for training a convolutional neural network. For this task you'll use the Keras ImageDataGenerator function to perform data preprocessing and data augmentation.


*   This class also provides support for basic data augmentation such as random horizontal flipping of images.
*   We also use the generator to transform the values in each batch so that their mean is 0 and their standard deviation is 1 (this will faciliate model training by standardizing the input distribution).
*   The generator also converts our single channel X-ray images (gray-scale) to a three-channel format by repeating the values in the image across all channels (we will want this because the pre-trained model that we'll use requires three-channel inputs).



In [ ]:
from keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)

Now we need to build a new generator for validation and t esting data.

In [ ]:
train = image_generator.flow_from_directory(train_dir, 
                                            shuffle=True, 
                                            class_mode='binary',
                                            target_size=(320, 320))

validation = image_generator.flow_from_directory(val_dir, 
                                                batch_size=1, 
                                                shuffle=False, 
                                                class_mode='binary',
                                                target_size=(320, 320))

test = image_generator.flow_from_directory(test_dir, 
                                            batch_size=1, 
                                            shuffle=False, 
                                            class_mode='binary',
                                            target_size=(320, 320))

## Model Building

**Class weights**

In [ ]:
weight_for_0 = num_pneumonia / (num_normal + num_pneumonia)
weight_for_1 = num_normal / (num_normal + num_pneumonia)

class_weight = {0: weight_for_0, 1: weight_for_1}

print(f"Weight for class 0: {weight_for_0:.2f}")
print(f"Weight for class 1: {weight_for_1:.2f}")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization


model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
CNN = model.fit(
    train, 
    epochs=10,
    validation_data=validation, 
    class_weight=class_weight,
    steps_per_epoch=100,
    validation_steps=12,
)


## Evaluation

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(CNN.history['loss'], label='Loss')
plt.plot(CNN.history['accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy and Loss Evolution')



In [ ]:
predicted_vals = model.predict(test, steps=len(test))

In [ ]:
print(confusion_matrix(test.classes, predicted_vals > 0.5))
pd.DataFrame(classification_report(test.classes, predicted_vals > 0.5, output_dict=True))

In [ ]:
print(confusion_matrix(test.classes, predicted_vals > 0.7))
pd.DataFrame(classification_report(test.classes, predicted_vals > 0.7, output_dict=True))

In [ ]:
evaluationCNN = model.evaluate(test)
evaluationCNNT = model.evaluate(train)


In [ ]:
evaluationCNN[1]

In [ ]:
evaluationCNNT[1]

Our model is ready! Now we will save it to Google Drive. For that, first we will mount the drive

# Transfer learning

## Introduction

Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.

Transfer learning is usually done for tasks where your dataset has too little data to train a full-scale model from scratch.

The most common incarnation of transfer learning in the context of deep learning is the following workflow:


1.   Take layers from a previously trained model.
2.   Freeze them, so as to avoid destroying any of the information they contain during future training rounds.
3.   Add some new, trainable layers on top of the frozen layers. They will learn to turn the old features into predictions on a new dataset.
4.   rain the new layers on your dataset.
A last, optional step, is fine-tuning, which consists of unfreezing the entire model you obtained above (or part of it), and re-training it on the new data with a very low learning rate. This can potentially achieve meaningful improvements, by incrementally adapting the pretrained features to the new data.



## VGG16

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

In [ ]:
from keras.applications.vgg16 import VGG16
taille=(320, 320, 3)
model_vgg16 = VGG16(input_shape=taille, include_top=False, weights='imagenet')

In [ ]:
model_vgg16.summary()

In [ ]:
# Freeze the base_model
model_vgg16.trainable = False

In [ ]:
layers = model_vgg16.layers
print(f"The model has {len(layers)} layers")

In [ ]:
model_vgg16.output

In [ ]:

x = model_vgg16.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
# Add a final sigmoid layer for classification
outputs =Dense(1, activation="sigmoid")(x)



In [ ]:
model_vgg16.summary()

In [ ]:
model = Model(inputs=model_vgg16.input, outputs=outputs)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


In [ ]:
vgg16 = model.fit(train,epochs=12,validation_data=validation,class_weight=class_weight,
    steps_per_epoch=100,validation_steps=25,
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(vgg16.history['loss'], label='Loss')
plt.plot(vgg16.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(vgg16.history['accuracy'], label='Accuracy')
plt.plot(vgg16.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
evaluationVGG16 = model.evaluate(test)
print(f"Test Accuracy: {evaluationVGG16[1] * 100:.2f}%")

evaluationVGG16T = model.evaluate(train)
print(f"Train Accuracy: {evaluationVGG16[1] * 100:.2f}%")

In [ ]:
evaluationVGG16[1]

In [ ]:
evaluationVGG16T[1]

## VGG19

In [ ]:
from keras.applications.vgg19 import VGG19

taille=(320, 320, 3)
model_vgg19 = VGG19(input_shape=taille, include_top=False, weights='imagenet')

In [ ]:
model_vgg19.summary()

In [ ]:
# Freeze the base_model
model_vgg19.trainable = False
layers = model_vgg19.layers
print(f"The model has {len(layers)} layers")

In [ ]:
model_vgg19.output

In [ ]:
x = model_vgg19.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
# Add a final sigmoid layer for classification
outputs =Dense(1, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=model_vgg19.input, outputs=outputs)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


In [ ]:
vgg19 = model.fit(train,epochs=12,validation_data=validation,class_weight=class_weight,
    steps_per_epoch=100,validation_steps=25,
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(vgg19.history['loss'], label='Loss')
plt.plot(vgg19.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(vgg19.history['accuracy'], label='Accuracy')
plt.plot(vgg19.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')


In [ ]:
evaluationVGG19 = model.evaluate(test)
print(f"Test Accuracy: {evaluationVGG19[1] * 100:.2f}%")

evaluationVGG19T = model.evaluate(train)
print(f"Train Accuracy: {evaluationVGG19[1] * 100:.2f}%")

In [ ]:
evaluationVGG19[1]

In [ ]:
evaluationVGG19T[1]

## InceptionV3

In [ ]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
taille=(320, 320, 3)
model_v3 =  InceptionV3(input_shape=taille, include_top=False, weights='imagenet')

In [ ]:
# Freeze the base_model
model_v3.trainable = False
layers = model_v3.layers
print(f"The model has {len(layers)} layers")

In [ ]:
model_v3.summary()

In [ ]:
model_v3.output

In [ ]:
x = model_v3.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
# Add a final sigmoid layer for classification
outputs =Dense(1, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=model_v3.input, outputs=outputs)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
V3 = model.fit(train,epochs=12,validation_data=validation,class_weight=class_weight,
    steps_per_epoch=100,validation_steps=25,
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(V3.history['loss'], label='Loss')
plt.plot(V3.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(V3.history['accuracy'], label='Accuracy')
plt.plot(V3.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
evaluationV3 = model.evaluate(test)
print(f"Test Accuracy: {evaluationV3[1] * 100:.2f}%")

evaluationV3T = model.evaluate(train)
print(f"Train Accuracy: {evaluationV3[1] * 100:.2f}%")

In [ ]:
evaluationV3[1]

In [ ]:
evaluationV3T[1]

## DenseNet

In [ ]:
from keras.applications import DenseNet121


In [ ]:
model_DenseNet121= DenseNet121(input_shape=taille, include_top=False, weights='imagenet')

In [ ]:
# Freeze the base_model
model_DenseNet121.trainable = False
layers = model_DenseNet121.layers
print(f"The model has {len(layers)} layers")

In [ ]:
model_DenseNet121.summary()

In [ ]:
model_DenseNet121.output

In [ ]:
x = model_DenseNet121.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
# Add a final sigmoid layer for classification
outputs =Dense(1, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=model_DenseNet121.input, outputs=outputs)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
DenseNet121 = model.fit(train,epochs=12,validation_data=validation,class_weight=class_weight,
    steps_per_epoch=100,validation_steps=25,
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(DenseNet121.history['loss'], label='Loss')
plt.plot(DenseNet121.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(DenseNet121.history['accuracy'], label='Accuracy')
plt.plot(DenseNet121.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
evaluationDenseNet121 = model.evaluate(test)
print(f"Test Accuracy: {evaluationDenseNet121[1] * 100:.2f}%")

evaluationDenseNet121T = model.evaluate(train)
print(f"Train Accuracy: {evaluationDenseNet121[1] * 100:.2f}%")

In [ ]:
evaluationDenseNet121[1]

In [ ]:
evaluationDenseNet121T[1]

## ResNet

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
model_ResNet= InceptionResNetV2(input_shape=taille, include_top=False, weights='imagenet')

In [ ]:
# Freeze the base_model
model_ResNet.trainable = False
layers = model_ResNet.layers
print(f"The model has {len(layers)} layers")

In [ ]:
model_ResNet.summary()

In [ ]:
model_ResNet.output

In [ ]:
x = model_ResNet.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
# Add a final sigmoid layer for classification
outputs =Dense(1, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=model_ResNet.input, outputs=outputs)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
ResNet = model.fit(train,epochs=12,validation_data=validation,class_weight=class_weight,
    steps_per_epoch=100,validation_steps=25,
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(ResNet.history['loss'], label='Loss')
plt.plot(ResNet.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(ResNet.history['accuracy'], label='Accuracy')
plt.plot(ResNet.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
evaluationResNet = model.evaluate(test)
print(f"Test Accuracy: {evaluationResNet[1] * 100:.2f}%")

evaluationResNetT = model.evaluate(train)
print(f"Train Accuracy: {evaluationResNet[1] * 100:.2f}%")

In [ ]:
evaluationResNet[1]

In [ ]:
evaluationResNetT[1]

## Xception

In [ ]:
from keras.applications import Xception

In [ ]:
model_Xception= Xception(input_shape=taille, include_top=False, weights='imagenet')

In [ ]:
# Freeze the base_model
model_Xception.trainable = False
layers = model_Xception.layers
print(f"The model has {len(layers)} layers")

In [ ]:
model_Xception.summary()

In [ ]:
model_Xception.output

In [ ]:
x = model_Xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
# Add a final sigmoid layer for classification
outputs =Dense(1, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=model_Xception.input, outputs=outputs)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
Xception = model.fit(train,epochs=12,validation_data=validation,class_weight=class_weight,
    steps_per_epoch=100,validation_steps=25,
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(Xception.history['loss'], label='Loss')
plt.plot(Xception.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(Xception.history['accuracy'], label='Accuracy')
plt.plot(Xception.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
evaluationXception = model.evaluate(test)
print(f"Test Accuracy: {evaluationXception[1] * 100:.2f}%")

evaluationXceptionT = model.evaluate(train)
print(f"Train Accuracy: {evaluationXception[1] * 100:.2f}%")

In [ ]:
evaluationXception[1]

In [ ]:
evaluationXceptionT[1]

In [ ]:
federatedT = 0.9196702241897583

In [ ]:
federated = 0.8028846383094788

# benchmarking

In [ ]:

colors = ["purple", "green", "orange", "magenta","#CFC60E","#0FBBAE","red","#CFC60E"]
X=['CNN' ,'VGG16','VGG19','InceptionV3','ResNet','DenseNet','Xception','federated']
Y=[evaluationCNNT[1],evaluationVGG16T[1],evaluationVGG19T[1],evaluationV3T[1],evaluationResNetT[1],evaluationDenseNet121T[1],evaluationXceptionT[1],federatedT]
sns.set_style("whitegrid")
plt.figure(figsize=(16,8))
plt.yticks(np.arange(0,100,10))
plt.ylabel("Precision %")
plt.xlabel("Algorithms DL")
plt.title("accuracy benchmark ")
g=sns.barplot(X, Y, palette=colors)
for i in range(len(Y)):
    g.text(i, Y[i],Y[i],color='black', ha="center")
plt.show()

In [ ]:

colors = ["purple", "green", "orange", "magenta","#CFC60E","#0FBBAE","red","#CFC60E"]
X=['CNN' ,'VGG16','VGG19','InceptionV3','ResNet','DenseNet','Xception','federated']
Y=[evaluationCNN[1],evaluationVGG16[1],evaluationVGG19[1],evaluationV3[1],evaluationResNet[1],evaluationDenseNet121[1],evaluationXception[1],federated]
sns.set_style("whitegrid")
plt.figure(figsize=(16,8))
plt.yticks(np.arange(0,100,10))
plt.ylabel("Precision %")
plt.xlabel("Algorithms DL")
plt.title("accuracy benchmark ")
g=sns.barplot(X, Y, palette=colors)
for i in range(len(Y)):
    g.text(i, Y[i],Y[i],color='black', ha="center")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)

plt.plot(CNN.history['loss'], label='Val_LossCNN')
plt.plot(vgg16.history['loss'], label='Val_Lossvgg16')
plt.plot(vgg19.history['loss'], label='Val_Lossvgg19')
plt.plot(V3.history['loss'], label='Val_LossV3')
plt.plot(ResNet.history['loss'], label='Val_LossResNet')
plt.plot(DenseNet121.history['loss'], label='Val_LossDenseNet121')
plt.plot(Xception.history['loss'], label='Val_LossXception')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(CNN.history['accuracy'], label='accuracyCNN')
plt.plot(vgg16.history['accuracy'], label='accuracyvgg16')
plt.plot(vgg19.history['accuracy'], label='accuracyvgg19')
plt.plot(V3.history['accuracy'], label='accuracyV3')
plt.plot(ResNet.history['accuracy'], label='accuracyResNet')
plt.plot(DenseNet121.history['accuracy'], label='accuracyDenseNet121')
plt.plot(Xception.history['accuracy'], label='accuracyXception')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)

plt.plot(CNN.history['loss'], label='Val_LossCNN')
plt.plot(vgg16.history['loss'], label='Val_Lossvgg16')
plt.plot(vgg19.history['loss'], label='Val_Lossvgg19')
plt.plot(V3.history['loss'], label='Val_LossV3')
plt.plot(ResNet.history['loss'], label='Val_LossResNet')
plt.plot(DenseNet121.history['loss'], label='Val_LossDenseNet121')
plt.plot(Xception.history['loss'], label='Val_LossXception')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(CNN.history['accuracy'], label='accuracyCNN')
plt.plot(vgg16.history['accuracy'], label='accuracyvgg16')
plt.plot(vgg19.history['accuracy'], label='accuracyvgg19')
plt.plot(V3.history['accuracy'], label='accuracyV3')
plt.plot(ResNet.history['accuracy'], label='accuracyResNet')
plt.plot(DenseNet121.history['accuracy'], label='accuracyDenseNet121')
plt.plot(Xception.history['accuracy'], label='accuracyXception')
plt.legend()
plt.title('Accuracy Evolution')